In [2]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [3]:
# Read in the data
data = pd.read_csv('used_cars_data.csv', index_col='vin')

F:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
data.count()

back_legroom            2840771
bed                       19568
bed_height               429098
bed_length               429098
body_type               2986497
                         ...   
wheel_system            2853308
wheel_system_display    2853308
wheelbase               2840771
width                   2840771
year                    3000040
Length: 65, dtype: int64

In [18]:
#Data cleanup
data = data.drop_duplicates().copy()

In [4]:
# Drop mostly null columns
percentage = 50
min_count =  int(((100-percentage)/100)*data.shape[0] + 1) 
dataDropped = data.dropna(axis=1, thresh=min_count)

In [20]:
# checking how many null values are in the dataframe that are over 0%
percentage = dataDropped.isna().sum()/dataDropped.shape[0]*100

pd.DataFrame(percentage[percentage > 0])

,0
back_legroom,5.308867
body_type,0.451400
city_fuel_economy,16.375933
description,2.596667
engine_cylinders,3.352667
engine_displacement,5.746100
engine_type,3.352667
exterior_color,0.000867
fleet,47.552633
frame_damaged,47.552633


In [ ]:
# Drop less useful rows
dataDropped.drop('description', axis=1, inplace=True)
dataDropped.drop('engine_cylinders', axis=1, inplace=True)
dataDropped.drop('engine_displacement', axis=1, inplace=True)
dataDropped.drop('engine_type', axis=1, inplace=True)
dataDropped.drop('exterior_color', axis=1, inplace=True)
dataDropped.drop('fleet', axis=1, inplace=True)
dataDropped.drop('frame_damaged', axis=1, inplace=True)
dataDropped.drop('has_accidents', axis=1, inplace=True)
dataDropped.drop('interior_color', axis=1, inplace=True)
dataDropped.drop('isCab', axis=1, inplace=True)
dataDropped.drop('franchise_make', axis=1, inplace=True)
dataDropped.drop('main_picture_url', axis=1, inplace=True)
dataDropped.drop('major_options', axis=1, inplace=True)
dataDropped.drop('salvage', axis=1, inplace=True)
dataDropped.drop('seller_rating', axis=1, inplace=True)
dataDropped.drop('sp_id', axis=1, inplace=True)
dataDropped.drop('theft_title', axis=1, inplace=True)
dataDropped.drop('transmission', axis=1, inplace=True)
dataDropped.drop('transmission_display', axis=1, inplace=True)
dataDropped.drop('trimId', axis=1, inplace=True)
dataDropped.drop('trim_name', axis=1, inplace=True)
dataDropped.drop('wheel_system_display', axis=1, inplace=True)
dataDropped.drop('power', axis=1, inplace=True)

In [ ]:
dataDropped.drop('city', axis=1, inplace=True)
dataDropped.drop('dealer_zip', axis=1, inplace=True)
dataDropped.drop('franchise_dealer', axis=1, inplace=True)
dataDropped.drop('is_new', axis=1, inplace=True)
dataDropped.drop('listed_date', axis=1, inplace=True)
dataDropped.drop('listing_color', axis=1, inplace=True)
dataDropped.drop('sp_name', axis=1, inplace=True)
dataDropped.drop('longitude', axis=1, inplace=True)
dataDropped.drop('latitude', axis=1, inplace=True)
dataDropped.drop('listing_id', axis=1, inplace=True)

In [30]:
# checking how many null values are in the dataframe that are over 0%
percentage = dataDropped.isna().sum()/dataDropped.shape[0]*100

pd.DataFrame(percentage[percentage > 0])

,0
back_legroom,5.308867
body_type,0.451400
city_fuel_economy,16.375933
front_legroom,5.308867
fuel_tank_volume,5.308867
fuel_type,2.757467
height,5.308867
highway_fuel_economy,16.375933
horsepower,5.746100
length,5.308867


In [ ]:
#Fill in the rest of the missing values with the mean/most common values

#back_legroom
dataDropped['back_legroom'] = dataDropped['back_legroom'].str.split(' ').str[0]
dataDropped['back_legroom'].replace('--', np.nan , inplace=True)
dataDropped['back_legroom'] = pd.to_numeric(dataDropped['back_legroom'], downcast='float')
dataDropped['back_legroom'].fillna(dataDropped['back_legroom'].mean(), inplace=True)

#body_type
dataDropped['body_type'].replace(np.nan, 'SUV / Crossover', inplace=True)

#fuel economy
dataDropped['city_fuel_economy'].replace(np.nan, dataDropped['city_fuel_economy'].mean(), inplace=True)
dataDropped['highway_fuel_economy'].fillna(dataDropped['highway_fuel_economy'].mean(), inplace=True)
dataDropped['combined_fuel_economy'] = (dataDropped['city_fuel_economy'] + dataDropped['highway_fuel_economy'])/2 
dataDropped.drop(['city_fuel_economy','highway_fuel_economy'], axis=1, inplace=True)

#front_legroom
dataDropped['front_legroom'] = dataDropped['front_legroom'].str.split(' ').str[0]
dataDropped['front_legroom'].replace("--", np.nan , inplace=True)
dataDropped['front_legroom'] = pd.to_numeric(dataDropped['front_legroom'], downcast='float')
dataDropped['front_legroom'].fillna(dataDropped['front_legroom'].mean(), inplace=True)

#fuel_tank type
dataDropped['fuel_type'].fillna("Gasoline", inplace=True)

#fuel_tank
dataDropped['fuel_tank_volume'] = dataDropped['fuel_tank_volume'].str.split(' ').str[0]
dataDropped['fuel_tank_volume'].replace("--", np.nan , inplace=True)
dataDropped['fuel_tank_volume'] = pd.to_numeric(dataDropped['fuel_tank_volume'], downcast='float')
dataDropped.loc[(dataDropped['fuel_type'] == "Electric"), 'fuel_tank_volume'] = 0
dataDropped['fuel_tank_volume'].fillna(dataDropped.groupby('fuel_type')['fuel_tank_volume'].transform('mean'), inplace=True)

#height
dataDropped['height'] = dataDropped['height'].str.split(' ').str[0]
dataDropped['height'].replace("--", np.nan , inplace=True)
dataDropped['height'] = pd.to_numeric(dataDropped['height'], downcast='float')
dataDropped['height'].fillna(dataDropped['height'].mean(), inplace=True)

#horsepower
dataDropped['horsepower'].fillna(dataDropped['horsepower'].mode()[0], inplace=True)

#length
dataDropped['length'] = dataDropped['length'].str.split(' ').str[0]
dataDropped['length'].replace("--", np.nan , inplace=True)
dataDropped['length'] = pd.to_numeric(dataDropped['length'], downcast='float')
dataDropped['length'].fillna(dataDropped['length'].median(), inplace=True)

#maximum_seating
dataDropped['maximum_seating'].replace("--", np.nan , inplace=True)
dataDropped['maximum_seating'].fillna("5 seats", inplace=True)
dataDropped['maximum_seating'] = dataDropped['maximum_seating'].str.replace(" seats", "")
dataDropped['maximum_seating'] = pd.to_numeric(dataDropped['maximum_seating'], downcast='integer')

#mileage
# using the interquartile range to remove outliers
q1 = dataDropped['mileage'].quantile(0.25)
q3 = dataDropped['mileage'].quantile(0.75)
# calculating the interquartile range
iqr = q3 - q1
# removing outliers
dataDropped = dataDropped[(dataDropped['mileage'] >= q1 - 1.5*iqr) & (dataDropped['mileage'] <= q3 + 1.5*iqr)]
dataDropped['mileage'].fillna(dataDropped['mileage'].mean(), inplace=True)

#torque
dataDropped['torque'] = dataDropped['torque'].str.split(" ").str[0]
dataDropped['torque'] = dataDropped['torque'].astype(float)
dataDropped['torque'].fillna(dataDropped['torque'].mean(), inplace=True)

#wheel_system
dataDropped['wheel_system'].fillna("FWD", inplace=True)

#wheel base
dataDropped['wheelbase'] = dataDropped['wheelbase'].str.split(" ").str[0]
dataDropped['wheelbase'].replace("--", np.nan, inplace=True)
dataDropped['wheelbase'] = dataDropped['wheelbase'].astype(float)
dataDropped['wheelbase'].fillna(dataDropped['wheelbase'].median(), inplace=True)

#width
dataDropped['width'] = dataDropped['width'].str.split(" ").str[0]
dataDropped['width'].replace("--", np.nan, inplace=True)
dataDropped['width'] = dataDropped['width'].astype(float)
dataDropped['width'].fillna(dataDropped['width'].mode()[0], inplace=True)

In [51]:
# instantiating the value counts
top_20 = dataDropped['model_name'].value_counts()

# creating a list of the top 20 models
top_20 = top_20[:20].index
 
# creating a function to return the top 20 models
def top_20_model(model):
    if model in top_20:
        return model
    else:
        return 'Other'
     
# applying the function to the column
dataDropped['model_name'] = dataDropped['model_name'].apply(top_20_model)

In [48]:
dataDropped.to_csv('cleaned_dataset.csv', index=True)

In [54]:
cat_col = dataDropped.select_dtypes(include='object').columns

for col in cat_col:
    print(col)
    dummies = pd.get_dummies(dataDropped[col], prefix=col, drop_first=True)
    dataDropped = pd.concat([dataDropped, dummies], axis=1)
    dataDropped.drop(col, axis=1, inplace=True)

Index([], dtype='object')


In [55]:
matrix = pd.DataFrame(dataDropped.corr()['price'].sort_values(ascending=False)).style.background_gradient(cmap='coolwarm')
matrix

,price
price,1.000000
horsepower,0.568886
torque,0.508020
fuel_tank_volume,0.409685
length,0.383464
wheelbase,0.376618
width,0.357801
height,0.316320
year,0.266295
back_legroom,0.257349


In [62]:
lessData = dataDropped.sample(n=50000, random_state=42)
lessData

,back_legroom,daysonmarket,front_legroom,fuel_tank_volume,height,horsepower,length,maximum_seating,mileage,price,...,model_name_Other,model_name_RAV4,model_name_Rogue,model_name_Silverado 1500,model_name_Trax,model_name_Tucson,wheel_system_4X2,wheel_system_AWD,wheel_system_FWD,wheel_system_RWD
vin,,,,,,,,,,,,,,,,,,,,,
WAUBFAFL8FN008628,35.200001,41,41.299999,16.1,56.200001,220.0,185.100006,5,31374.0,18470.0,...,1,0,0,0,0,0,0,1,0,0
1VWBS7A33EC119460,39.099998,25,42.400002,18.5,58.500000,170.0,191.600006,5,80800.0,11500.0,...,1,0,0,0,0,0,0,0,1,0
1C6RR7KG5HS507094,40.299999,40,41.000000,26.0,77.500000,305.0,229.000000,6,57943.0,24995.0,...,0,0,0,0,0,0,0,0,0,0
4T1C11AK0LU383298,38.000000,14,42.099998,16.0,56.900002,203.0,192.100006,5,10.0,22715.0,...,0,0,0,0,0,0,0,0,1,0
JTEBU5JR0L5836692,32.900002,7,41.700001,23.0,71.500000,270.0,191.300003,5,0.0,43339.0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1HGCV1F55JA065771,40.400002,12,42.299999,14.8,57.099998,192.0,192.199997,5,29155.0,23990.0,...,0,0,0,0,0,0,0,0,1,0
1GKKNKLA4HZ238603,39.700001,45,41.000000,19.0,66.000000,193.0,193.600006,7,32987.0,21800.0,...,1,0,0,0,0,0,0,0,1,0
KL79MPSL5MB003365,39.400002,146,40.900002,13.2,65.199997,155.0,173.500000,5,129.0,24758.0,...,1,0,0,0,0,0,0,0,1,0


In [66]:
dataDropped.to_csv('cleaned_dataset_dummy.csv', index=True)

In [67]:
lessData.to_csv('cleaned_dataset_shorten.csv', index=True)

In [70]:
x = lessData.drop('price', axis = 1)
y = lessData['price']

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

forestModel = RandomForestRegressor(n_estimators=100, random_state=42)
forestModel.fit(xtrain, ytrain)

linearModel = LinearRegression()
linearModel.fit(xtrain, ytrain)

LinearRegression()

In [74]:
forestPredictions = forestModel.predict(xtest)
forestMAE = mean_absolute_error(ytest, forestPredictions)
print('Mean Absolute Error for Random Forest:', forestMAE)

linearPredictions = linearModel.predict(xtest)
linearMAE = mean_absolute_error(ytest, linearPredictions)
print('Mean Absolute Error for Linear Regression:', linearMAE)

Mean Absolute Error for Random Forest: 2956.6777851278707
Mean Absolute Error for Linear Regression: 5544.367054850979
